In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

exper = anndata.read_h5ad(save_path)
exper

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
from scdesigner.simulators import CompositeGLMSimulator

specification = {
    "group1": {"formula": "~ pseudotime", "distribution": "negbin", "var_names": exper.var_names[:50]},
    "group2": {"formula": "~ 1", "distribution": "poisson", "var_names": exper.var_names[50:]}
}

sim = CompositeGLMSimulator(specification, epochs=10)
sim.fit(exper)

In [3]:
sim

scDesigner simulator object with
    method: 'Composite'
    specification: {'group1': ('negbin', '~ pseudotime', '[Pyy,Iapp, ..., Serpina1c]'), 'group2': ('poisson', '~ 1', '[Dbpht2,Krt18, ..., 1110012L19Rik]')}
    parameters: ['group1.beta', 'group1.gamma', 'group2.beta']

In [4]:
sim.predict(exper.obs)

{'group1': {'mean':                         Pyy       Iapp       Chgb       Rbp4      Spp1  \
  AAACCTGAGAGGGATA  20.443543  18.265802  19.472219  16.468190  4.819735   
  AAACCTGGTAAGTGGC  13.088421  11.861833  12.523801  10.761526  5.731384   
  AAACGGGCAAAGAATC  23.303660  20.734400  22.166605  18.659560  4.580692   
  AAACGGGGTACAGTTC  32.079277  28.252794  30.414010  25.311946  4.045852   
  AAACGGGGTGAAATCA  11.926304  10.840752  11.422722   9.847989  5.942194   
  ...                     ...        ...        ...        ...       ...   
  TTTGGTTTCACTTACT   7.215083   6.664412   6.946201   6.096910  7.223340   
  TTTGGTTTCCTTTCGG  25.222668  22.385221  23.972482  20.122851  4.442018   
  TTTGTCAAGAATGTGT  21.268062  18.978521  20.249335  17.101286  4.746268   
  TTTGTCAAGTGACATA  17.472379  15.689595  16.669094  14.176694  5.122937   
  TTTGTCAAGTGTGGCA  10.812408   9.859053  10.366296   8.968511  6.172905   
  
                         Chga        Cck       Ins1       Nnat     

In [5]:
sim.sample(exper.obs)

AnnData object with n_obs × n_vars = 2087 × 100